## import

In [2]:
import os
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from xgboost import plot_importance

from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

## Data Load

In [4]:
# 파일 호출
data_path = "../data"
train_df = pd.read_csv(os.path.join(data_path, "train.csv"))
test_df = pd.read_csv(os.path.join(data_path, "test.csv"))
sample_submission = pd.read_csv(os.path.join(data_path, "sample_submission.csv"))

# sub data
interestrate_df = pd.read_csv(os.path.join(data_path, "interestRate.csv"))
park_df = pd.read_csv(os.path.join(data_path, "parkInfo.csv"))
school_df = pd.read_csv(os.path.join(data_path, "schoolinfo.csv"))
subway_df = pd.read_csv(os.path.join(data_path, "subwayInfo.csv"))

## feature extraction

#### park, school, subway

In [4]:
places_dict = {'park':park_df, 'school':school_df, 'subway':subway_df}
radii_dict = {
    'park': [500, 1000, 1500, 2000],
    'school': [100, 200, 300, 400, 500, 1000, 1500, 2000],
    'subway': [300, 500, 800, 1000, 1500, 2000]
}

In [5]:
from sklearn.neighbors import BallTree

# 위경도를 라디안으로 변환
def to_radians(df, lat_col='latitude', lon_col='longitude'):
    df['latitude_radi'] = np.radians(df[lat_col])
    df['longitude_radi'] = np.radians(df[lon_col])
    return df

In [6]:
def count_places(main_df, places_dict, radii_dict):
    """
    main_df: 원본 데이터 (계산 대상 위치)
    places_dict: 장소 데이터 딕셔너리, {장소 이름: 장소 데이터프레임}
    radii_dict: 반경 리스트 딕셔너리, {장소 이름: 반경 리스트(예: [300, 500, 1000])}
    """
    
    # 위경도를 라디안으로 변환 (main_df에 적용)
    main_df = to_radians(main_df)

    # 각 장소 유형별로 BallTree를 생성하여 반경 내 개수를 계산
    for place_name, add_df in places_dict.items():

        # 각 장소 데이터의 위경도를 라디안으로 변환
        add_df = to_radians(add_df)

        # 장소 데이터에 대해 BallTree 생성
        ball_tree = BallTree(add_df[['latitude_radi', 'longitude_radi']].values, metric='haversine')

        for radius in radii_dict[place_name]:
            # 반경을 km로 변환
            radius_in_km = radius / 1000

            # 반경 내 place 개수 계산(반경을 지구 반지름(6371km)로 나눈 값)
            _, indices = ball_tree.query_radius(main_df[['latitude_radi', 'longitude_radi']].values, r=radius_in_km/6371, return_distance=True)

            # 반경별 장소 개수 열 추가
            main_df[f'{place_name}_{radius}m'] = [len(idx) for idx in indices]
        
        distances, distances_index = ball_tree.query(main_df[['latitude_radi', 'longitude_radi']].values, k=1)
        main_df[f'{place_name}_near_distance'] = distances.flatten()*6371

        # 공원: area 추가
        if place_name == 'park':
            main_df['park_near_area'] = add_df.iloc[distances_index.flatten(), 2].reset_index(drop=True)

    return main_df

In [8]:
train_df = count_places(train_df, places_dict, radii_dict)
train_df.to_csv("train_df_seperate.csv")
test_df = count_places(test_df, places_dict, radii_dict)
test_df.to_csv("test_df_seperate.csv")

### Data load

In [5]:
train_df = pd.read_csv("train_df.csv")
test_df = pd.read_csv("test_df.csv")

## feature selection

In [6]:
train_df['age'] = train_df['age'].clip(lower=0)

age_mapping = {'0-10': 0, '10-20': 10, '20-30': 20, '30+': 30}
train_df['age_group'] = pd.cut(train_df['age'], bins=[0, 10, 20, 30, 300], labels=[0, 10, 20, 30], right=False)
train_df['age_group'] = train_df['age_group'].astype(int)
test_df['age_group'] = pd.cut(test_df['age'], bins=[0, 10, 20, 30, 300], labels=[0, 10, 20, 30], right=False)
test_df['age_group'] = test_df['age_group'].astype(int)

In [7]:
train_df = train_df.drop(columns=['index', 'contract_day', 'age'])
test_df = test_df.drop(columns=['index', 'contract_day', 'age'])

In [8]:
train_df['contract_type'] = train_df['contract_type'].replace(2, np.nan)
test_df['contract_type'] = test_df['contract_type'].replace(2, np.nan)

### add feature : deposit per area

In [9]:
train_df['deposit_per_area'] = train_df['deposit'] / train_df['area_m2']

In [10]:
train_df_copy = train_df

y_valid_real = train_df_copy[(train_df_copy['contract_year_month'] >= 202307) & (train_df_copy['contract_year_month'] <= 202312)]

y_valid_real_deposit = y_valid_real['deposit']

In [11]:
train_df.drop(columns=['deposit'], inplace=True)

## train, valid split

In [12]:
holdout_start = 202307
holdout_end = 202312
valid_df = train_df[(train_df['contract_year_month'] >= holdout_start) & (train_df['contract_year_month'] <= holdout_end)]
final_train_df = train_df[~((train_df['contract_year_month'] >= holdout_start) & (train_df['contract_year_month'] <= holdout_end))]

## X, y split

In [13]:
X_train = final_train_df.drop(columns=['deposit_per_area'])
y_train = final_train_df['deposit_per_area']
X_valid = valid_df.drop(columns=['deposit_per_area'])
y_valid = valid_df['deposit_per_area']
X_test = test_df.copy()

X_total = train_df.drop(columns=['deposit_per_area'])
y_total = train_df['deposit_per_area']

## Custom Loss

In [14]:
class AsymmetricHuberLoss:
    def __init__(self, delta=1.0, beta=1.05):   # 작게 예측하는 값에 대해 loss를 키운다.(default : 5%)
        self.delta = delta
        self.beta = beta

    def _calculate_loss(self, y_true, y_pred):

        # 식 구현
        error = y_true - y_pred
        abs_error = np.abs(error) / y_true
        quadratic = np.minimum(abs_error, self.delta)   
        linear = abs_error - quadratic 
        loss = 0.5 * quadratic**2 + self.delta * linear

        # loss penalty 추가
        underestimation_mask = y_pred < y_true * 0.95 
        loss[underestimation_mask] *= self.beta
        return loss   # eval_metric에 사용

    def gradient(self, y_true, y_pred):   # 1차 미분값
        error = y_pred - y_true
        abs_error = np.abs(error) / y_true
        grad = np.where(abs_error <= self.delta, error, self.delta * np.sign(error))  
        grad[y_pred < y_true * 0.95] *= self.beta   
        return grad

    def hessian(self, y_true, y_pred):   # 2차 미분값
        abs_error = np.abs(y_pred - y_true) / y_true
        hess = np.where(abs_error <= self.delta, 1.0, 0.0)
        hess[y_pred < y_true * 0.95] *= self.beta   
        return hess

def custom_loss(y_true, y_pred):
    loss = AsymmetricHuberLoss()
    grad = loss.gradient(y_true, y_pred)
    hess = loss.hessian(y_true, y_pred)
    return grad, hess

def custom_metric(y_pred, y_true):
    loss = AsymmetricHuberLoss()
    return np.mean(loss._calculate_loss(y_true, y_pred)) * 1000

## 군집화

In [15]:
from sklearn.cluster import KMeans

In [16]:
# train + valid 데이터로 최적의 k 찾기
best_k = 10
kmeans = KMeans(n_clusters=best_k, random_state=RANDOM_SEED)
kmeans.fit(X_total[['latitude', 'longitude']])
total_pred = kmeans.predict(X_total[['latitude', 'longitude']])

/opt/conda/envs/house/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [17]:
total_pred

array([4, 4, 4, ..., 5, 5, 5], dtype=int32)

In [18]:
# test 데이터에 대한 cluster 예측
test_pred = kmeans.predict(X_test[['latitude', 'longitude']])

## 모델링

#### xgboost

In [19]:
xgb_models = []
best_iterations = []
train_pred = kmeans.predict(X_train[['latitude', 'longitude']])
valid_pred = kmeans.predict(X_valid[['latitude', 'longitude']])
X_train = X_train.drop(columns=['latitude', 'longitude'])
X_valid = X_valid.drop(columns=['latitude', 'longitude'])

In [20]:
for i in range(best_k):
    print(f'Cluster {i} modeling...')
    train_cluster_idx = np.where(train_pred == i)[0]   # (index_array, dtype)
    valid_cluster_idx = np.where(valid_pred == i)[0]

    X_train_cluster = X_train.iloc[train_cluster_idx]
    y_train_cluster = y_train.iloc[train_cluster_idx]

    X_valid_cluster = X_valid.iloc[valid_cluster_idx]
    y_valid_cluster = y_valid.iloc[valid_cluster_idx]

    xgb_params = {
        'objective': custom_loss,   # default : reg:squarederror
        'eval_metric': custom_metric,   # default : rmse
        'seed': RANDOM_SEED,
        'n_estimators': 500,
        'learning_rate': 0.02,   # default : 0.3
        'max_depth': 12,
        # 'subsample': 0.9,
        # 'colsample_bytree': 0.9,
        # 'reg_alpha': 10.0,
        # 'reg_lambda': 10.0,
        'early_stopping_rounds':20,
        'n_jobs': -1,
        'device': 'cuda'
    }

    xgb_model = xgb.XGBRegressor(**xgb_params)
    xgb_model.fit(X_train_cluster, y_train_cluster, eval_set=[(X_valid_cluster, y_valid_cluster)], verbose=50)
    best_iterations.append(xgb_model.best_iteration)

    xgb_models.append(xgb_model)

Cluster 0 modeling...
[0]	validation_0-rmse:531.25830	validation_0-custom_metric:48911.09085
[50]	validation_0-rmse:225.61841	validation_0-custom_metric:233.06107
[100]	validation_0-rmse:121.61232	validation_0-custom_metric:37.64549
[150]	validation_0-rmse:90.26400	validation_0-custom_metric:17.91112
[200]	validation_0-rmse:81.28798	validation_0-custom_metric:13.98423
[250]	validation_0-rmse:78.58293	validation_0-custom_metric:12.90064
[300]	validation_0-rmse:77.77264	validation_0-custom_metric:12.56116
[350]	validation_0-rmse:77.48353	validation_0-custom_metric:12.46947
[373]	validation_0-rmse:77.47522	validation_0-custom_metric:12.49817
Cluster 1 modeling...
[0]	validation_0-rmse:518.57477	validation_0-custom_metric:49467.11731
[50]	validation_0-rmse:220.86951	validation_0-custom_metric:248.38427
[100]	validation_0-rmse:117.75989	validation_0-custom_metric:39.87058
[150]	validation_0-rmse:84.41791	validation_0-custom_metric:18.00197
[200]	validation_0-rmse:74.36652	validation_0-custo

In [21]:
# for xgb_model in xgb_models:
#     plt.figure(figsize=(10, 8))
#     plot_importance(xgb_model, importance_type='gain')  # max_num_features로 출력할 feature 개수 지정 가능
#     plt.title('Feature Importance')
#     plt.show()

## valid 성능

In [22]:
y_valid_real_deposit.info()

<class 'pandas.core.series.Series'>
Index: 206866 entries, 774291 to 1801227
Series name: deposit
Non-Null Count   Dtype  
--------------   -----  
206866 non-null  float64
dtypes: float64(1)
memory usage: 3.2 MB


In [24]:
X_valid['pred'] = 0
for i in range(best_k):
    valid_cluster_idx = np.where(valid_pred == i)[0]
    X_valid_cluster = X_valid.iloc[valid_cluster_idx]
    X_valid.loc[X_valid_cluster.index, 'pred'] = xgb_models[i].predict(X_valid_cluster.drop(columns=['pred']))

valid_pred = X_valid['pred'] * X_valid['area_m2']
valid_mae = mean_absolute_error(y_valid_real_deposit, valid_pred)

print(valid_mae)

/opt/conda/envs/house/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [01:21:12] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


4240.15151856872


## 최종 학습

In [70]:
xgb_models = []
total_pred = kmeans.predict(X_total[['latitude', 'longitude']])
X_total = X_total.drop(columns=['latitude', 'longitude'])
for i in range(best_k):
    print(f'Cluster {i} modeling...')
    total_cluster_idx = np.where(total_pred == i)[0]   # (index_array, dtype)

    X_total_cluster = X_total.iloc[total_cluster_idx]
    y_total_cluster = y_total.iloc[total_cluster_idx]

    xgb_params = {
        'objective': custom_loss,
        'eval_metric': custom_metric,
        'seed': RANDOM_SEED,
        'n_estimators': round(best_iterations[i], -1),   # 1의 자리에서 반올림
        'learning_rate': 0.02,   # default : 0.3
        'max_depth': 12,
        # 'subsample': 0.9,
        # 'colsample_bytree': 0.9,
        # 'reg_alpha': 10.0,
        # 'reg_lambda': 10.0,
        # 'early_stopping_rounds':20,
        'n_jobs': -1,
        'device': 'cuda'
    }

    xgb_model = xgb.XGBRegressor(**xgb_params)
    xgb_model.fit(X_total_cluster, y_total_cluster, verbose=20)

    xgb_models.append(xgb_model)

Cluster 0 modeling...
Cluster 1 modeling...
Cluster 2 modeling...
Cluster 3 modeling...
Cluster 4 modeling...
Cluster 5 modeling...
Cluster 6 modeling...
Cluster 7 modeling...
Cluster 8 modeling...
Cluster 9 modeling...


## submit

In [71]:
X_test['pred'] = 0
X_test = X_test.drop(columns=['latitude', 'longitude'])
for i in range(best_k):
    test_cluster_idx = np.where(test_pred == i)[0]
    X_test_cluster = X_test.iloc[test_cluster_idx]
    X_test.loc[X_test_cluster.index, 'pred'] = xgb_models[i].predict(X_test_cluster.drop(columns=['pred']))

In [72]:
test_pred_xgb_cluster = X_test['pred'] * X_test['area_m2']

In [73]:
sample_submission['deposit'] = test_pred_xgb_cluster
sample_submission.to_csv('output.csv', index=False, encoding='utf-8-sig')